Imports

In [ ]:
import face_recognition
import os, sys
import cv2
import numpy as np
import math
from PIL import Image

Testing The Camera

In [ ]:
camera = cv2.VideoCapture(0)
camera .set(3,640) #WIDTH
camera .set(4,480) #HEIGHT
while(True):
    ret, frame = camera .read()
    cv2.imshow('frame', frame)
    key = cv2.waitKey(30) & 0xff
    if key == 27: 
        break
camera .release()
cv2.destroyAllWindows()

Gathering data

In [ ]:
directory = r'Directory to store frames captured' 
os.chdir(directory)

In [ ]:
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')
camera = cv2.VideoCapture(0)
camera .set(3,640) #WIDTH
camera .set(4,480) #HEIGHT
sampleCount = 0
name = input('Enter Person Name')
while True:
    ret, img = camera.read()
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    faces = faceCascade.detectMultiScale(
        gray,     
        scaleFactor=1.2,
        minNeighbors=5, #higher number gives lower false positives    
        minSize=(20, 20))
    
    for (x,y,w,h) in faces: #Rectangle Cordinates
        cv2.rectangle(img,(x,y),(x+w,y+h),(255,0,0),2)
        sampleCount+=1
        roi_gray = gray[y:y+h, x:x+w]
        roi_color = img[y:y+h, x:x+w] 

        # Save the captured frames
        filename = str(name) + '.' +  str(sampleCount) + ".jpg"
        cv2.imwrite(filename, gray[y:y+h,x:x+w])

    cv2.imshow('video',img)
    key = cv2.waitKey(5) & 0xff
    if key == 27: # press 'ESC' to quit
        break

camera.release()
cv2.destroyAllWindows()

In [ ]:
def getImagesAndLabels(path):
    imagePaths = [os.path.join(path,f) for f in os.listdir(path)]     
    faceSamples=[]
    labels = []
    for imagePath in imagePaths:
        PIL_img = Image.open(imagePath).convert('L')
        img_numpy = np.array(PIL_img,'uint8')
        label = int(os.path.split(imagePath)[-1].split(".")[1])
        faces = faceCascade.detectMultiScale(img_numpy)
        for (x,y,w,h) in faces:
            faceSamples.append(img_numpy[y:y+h,x:x+w])
            labels.append(label)

    return faceSamples,labels

In [ ]:
# Path for the database
path = 'Directory which has all frames'
faceRecognizer = cv2.face.LBPHFaceRecognizer_create()
faceCascade = cv2.CascadeClassifier(cv2.data.haarcascades +'haarcascade_frontalface_default.xml')

In [ ]:
faces,labels = getImagesAndLabels(path)
faceRecognizer.train(faces, np.array(labels))
faceRecognizer.write('Training.yml')

In [ ]:
faceRecognizer.read('Training.yml')
font = cv2.FONT_HERSHEY_SIMPLEX

In [ ]:
camera = cv2.VideoCapture(0)
camera.set(3, 640) 
camera.set(4, 480) 
minW = 0.1*camera.get(3)
minH = 0.1*camera.get(4)

while True:

    ret, img =camera.read()
    gray = cv2.cvtColor(img,cv2.COLOR_BGR2GRAY)

    faces = faceCascade.detectMultiScale( 
        gray,
        scaleFactor = 1.2,
        minNeighbors = 5,
        minSize = (int(minW), int(minH)),
       )

    for(x,y,w,h) in faces:

        cv2.rectangle(img, (x,y), (x+w,y+h), (0,255,0), 2)

        id, confidence = faceRecognizer.predict(gray[y:y+h,x:x+w])

        # Check if confidence is less them 100 ==> "0" is perfect match 
        if (confidence < 100):
            id = labels[id]
            confidence = "  {0}%".format(round(100 - confidence))
        else:
            id = "unknown"
            confidence = "  {0}%".format(round(100 - confidence))
        
        cv2.putText(img, str(id), (x+5,y-5), font, 1, (255,255,255), 2)
        cv2.putText(img, str(confidence), (x+5,y+h-5), font, 1, (255,255,0), 1)  
    
    cv2.imshow('camera',img) 

    k = cv2.waitKey(10) & 0xff # Press 'ESC' for exiting video
    if k == 27:
        break

camera.release()
cv2.destroyAllWindows()